In [2]:
import json
import numpy as np

### With softmax and log likelyhood calculated in BO

In [6]:
with open("../BayesianOptimization/results.json", "r") as f:
    res = json.load(f)

T = len(res)

ll_EI = sum(r["logp_EI"] for r in res) / T
ll_PI = sum(r["logp_PI"] for r in res) / T
ll_UCB = sum(r["logp_UCB"] for r in res) / T
baseline = np.log(1/10000)

print(f"Random baseline: {baseline:.4f}")
print(f"EI: {ll_EI:.4f}")
print(f"PI: {ll_PI:.4f}")
print(f"UCB: {ll_UCB:.4f}")

Random baseline: -9.2103
EI: -9.2103
PI: -9.2445
UCB: -9.0413


### If they were not calculated in BO

In [ ]:
def softmax(x):
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x)

# frim x,y coordinates to flat index in 100x100 grid
def xy_to_flat_index(x, y):
    xi = int(np.clip(np.floor(x * 100), 0, 99))
    yi = int(np.clip(np.floor(y * 100), 0, 99))
    return yi * 100 + xi

In [ ]:
# Load results from File 1
with open("../BayesianOptimization/results.json", "r") as f:
    loaded_results = json.load(f)

# Convert lists back to numpy arrays
for r in loaded_results:
    r["all_EI_values"] = np.array(r["all_EI_values"])
    r["all_PI_values"] = np.array(r["all_PI_values"])
    r["all_UCB_values"] = np.array(r["all_UCB_values"])

beta = 6

ll_EI = 0.0
ll_PI = 0.0
ll_UCB = 0.0

# Compute log-likelihoods for each acquisition function
for r in loaded_results:
    xh, yh = r["human_xy"]
    idx = xy_to_flat_index(xh, yh)

    P_ei  = softmax(beta * r["all_EI_values"])
    P_pi  = softmax(beta * r["all_PI_values"])
    P_ucb = softmax(beta * r["all_UCB_values"])

    ll_EI  += np.log(P_ei[idx]  + 1e-12)
    ll_PI  += np.log(P_pi[idx]  + 1e-12)
    ll_UCB += np.log(P_ucb[idx] + 1e-12)

print("Log-likelihoods:")
print("EI:", ll_EI)
print("PI:", ll_PI)
print("UCB:", ll_UCB)